In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import json
import torch
from transformers import BertConfig
from fastformer import FastformerEncoder

In [4]:
config=BertConfig.from_json_file('fastformer.json')

In [6]:
model = FastformerEncoder(config)

In [8]:
batch_size = 8
seq_len = 50 
dummy_input = torch.randn(batch_size, seq_len, config.hidden_size)
dummy_mask = torch.ones(batch_size, seq_len)

In [9]:
model.eval()
with torch.no_grad():
    output = model(dummy_input, dummy_mask)

In [11]:
print(dummy_input.shape)
print(output.shape)

torch.Size([8, 50, 256])
torch.Size([8, 256])
